In [9]:
from dotenv import load_dotenv
from langchain.retrievers import ChatGPTPluginRetriever
from langchain.chat_models import AzureChatOpenAI
from langchain.tools.plugin import AIPlugin
from langchain.agents import (
    Tool,
    AgentExecutor,
    LLMSingleActionAgent,
    AgentOutputParser,
)
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

import openai
import os
import requests

# .env file
# DATASTORE=
# BEARER_TOKEN=
# OPENAI_API_KEY=
# OPENAI_API_BASE=
# OPENAI_API_TYPE=
# OPENAI_EMBEDDINGMODEL_DEPLOYMENTID=
# OPENAI_METADATA_EXTRACTIONMODEL_DEPLOYMENTID=
# OPENAI_COMPLETIONMODEL_DEPLOYMENTID=

load_dotenv()

True

In [10]:
OPENAI_API_VERSION = "2023-03-15-preview"
OPENAI_MODEL_NAME =  "gpt-35-turbo-0301"  # os.getenv("OPENAI_COMPLETIONMODEL_DEPLOYMENTID")
PLUGIN_ENDPOINT_URL = "http://0.0.0.0:3333"
PLUGIN_BEARER_TOKEN = os.getenv("BEARER_TOKEN")
PLUGIN_HEADERS = {
    "Authorization": f"Bearer {PLUGIN_BEARER_TOKEN}"
}

openai.api_type = os.getenv('OPENAI_API_TYPE')
openai.api_version = OPENAI_API_VERSION
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

In [11]:
openai.Deployment.list()

<OpenAIObject list at 0x11751c6d0> JSON: {
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-embedding-ada-002",
      "owner": "organization-owner",
      "id": "text-embedding-ada-002",
      "status": "succeeded",
      "created_at": 1678902936,
      "updated_at": 1678902936,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "gpt-35-turbo",
      "owner": "organization-owner",
      "id": "gpt-35-turbo-0301",
      "status": "succeeded",
      "created_at": 1680520260,
      "updated_at": 1689939708,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-embedding-ada-002",
      "owner": "organization-owner",
      "id": "text-embedding-ada-002-v2",
      "status": "succeeded",
      "created_at": 1683751121,
      "updated_at": 1683751121,
      "object": "deployment"
    },


In [12]:
llm = AzureChatOpenAI(deployment_name=OPENAI_MODEL_NAME, openai_api_version=OPENAI_API_VERSION)

In [13]:
plugin = AIPlugin.from_url(f"{PLUGIN_ENDPOINT_URL}/.well-known/ai-plugin.json")
plugin_retriever = ChatGPTPluginRetriever(url=PLUGIN_ENDPOINT_URL, bearer_token=PLUGIN_BEARER_TOKEN)
plugin_description = plugin.description_for_model
plugin_name = plugin.name_for_model

In [14]:
# ADD DOCUMENTS ABOUT Alice Phone Number

In [15]:
document_1 = {
    "id": "twtr",
    "text": """This is Alice's phone number: 123-456-7890 Email: myemail@mail.com""",
    "metadata" : {
        "source" : "file",
        "source_id" : "test",
        "created_at": "2020-12-31",
        "author": 'Alice'        
    }
}

In [16]:
response = requests.post(
    f"{PLUGIN_ENDPOINT_URL}/upsert",
    headers=PLUGIN_HEADERS,
    json={
        "documents": [document_1]
    }
)
response.raise_for_status()

In [17]:
query = {
    "query": "What is alice phone number?",
    # "filter": {"source_id": "test:twtr10k|test:tesla10k"},
    "top_k": 3
}

response = requests.post(
    f"{PLUGIN_ENDPOINT_URL}/query",
    headers=PLUGIN_HEADERS,
    json={
        "queries": [query]
    }
)
response.raise_for_status()

response.json()

{'results': [{'query': 'What is alice phone number?',
   'results': [{'id': 'twtr_0',
     'text': "This is Alice's phone number: 123-456-7890 Email: myemail@mail.com",
     'metadata': {'source': 'file',
      'source_id': 'test',
      'url': None,
      'created_at': '2020-12-31',
      'author': 'Alice',
      'document_id': 'twtr'},
     'embedding': None,
     'score': 0.9151652688749533}]}]}

In [18]:
def top_3_selector(query):
    ret = [doc.page_content for doc in plugin_retriever.get_relevant_documents(query)[:3]]
    return '\n'.join(ret)


print(top_3_selector("alice's phone number"))

This is Alice's phone number: 123-456-7890 Email: myemail@mail.com


In [19]:
# PLUGIN SELECTOR
# https://python.langchain.com/docs/modules/agents/how_to/custom_agent_with_tool_retrieval

from langchain.agents import (
    Tool,
    AgentExecutor,
    LLMSingleActionAgent,
    AgentOutputParser,
)
from langchain.prompts import StringPromptTemplate
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

In [20]:
# Define which tools the agent can use to answer user queries

# DEFINE TELEPHONE SELECTOR :P 
plugin_tool = Tool(
    name=plugin_name,
    func=top_3_selector,
    description=plugin_description
)


def fake_func(inp: str) -> str:
    return "foo"


fake_tools = [
    Tool(
        name=f"foo-{i}",
        func=fake_func,
        description=f"a silly function that you can use to get more information about the {i}",
    )
    for i in range(10)
]
ALL_TOOLS = [plugin_tool] + fake_tools

In [21]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

In [22]:
docs = [
    Document(page_content=t.description, metadata={"index": i})
    for i, t in enumerate(ALL_TOOLS)
]

In [23]:
docs

[Document(page_content="Plugin for searching through the user's documents (such as files, emails, phone numbers and more) to find answers to questions and retrieve relevant information. Use it whenever a user asks something that might be found in their personal information.", metadata={'index': 0}),
 Document(page_content='a silly function that you can use to get more information about the 0', metadata={'index': 1}),
 Document(page_content='a silly function that you can use to get more information about the 1', metadata={'index': 2}),
 Document(page_content='a silly function that you can use to get more information about the 2', metadata={'index': 3}),
 Document(page_content='a silly function that you can use to get more information about the 3', metadata={'index': 4}),
 Document(page_content='a silly function that you can use to get more information about the 4', metadata={'index': 5}),
 Document(page_content='a silly function that you can use to get more information about the 5', met

In [24]:
vector_store = FAISS.from_documents(docs, OpenAIEmbeddings(deployment="text-embedding-ada-002-v2"))

In [25]:
retriever = vector_store.as_retriever()

def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [ALL_TOOLS[d.metadata["index"]] for d in docs]

In [26]:
get_tools("what is the phone number of Alice?")

[Tool(name='foo-9', description='a silly function that you can use to get more information about the 9', args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x1173cb4f0>, func=<function fake_func at 0x1174d2440>, coroutine=None),
 Tool(name='foo-1', description='a silly function that you can use to get more information about the 1', args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x1173cb4f0>, func=<function fake_func at 0x1174d2440>, coroutine=None),
 Tool(name='foo-6', description='a silly function that you can use to get more information about the 6', args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x1173cb4f0>, func=<function fake_func at 0x1174d2440>, coroutine=None),
 Tool(name='foo-2', description='a silly function that you can use to get

In [27]:
get_tools("whats the files?")

[Tool(name='retrieval', description="Plugin for searching through the user's documents (such as files, emails, phone numbers and more) to find answers to questions and retrieve relevant information. Use it whenever a user asks something that might be found in their personal information.", args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x1173cb4f0>, func=<function top_3_selector at 0x117960a60>, coroutine=None),
 Tool(name='foo-5', description='a silly function that you can use to get more information about the 5', args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x1173cb4f0>, func=<function fake_func at 0x1174d2440>, coroutine=None),
 Tool(name='foo-4', description='a silly function that you can use to get more information about the 4', args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callb

In [28]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""

In [29]:
from typing import Callable


# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    ############## NEW ######################
    # The list of tools available
    tools_getter: Callable

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        tools = self.tools_getter(kwargs["input"])
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in tools]
        )
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

In [30]:
prompt = CustomPromptTemplate(
    template=template,
    tools_getter=get_tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"],
)

In [31]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )

In [32]:
output_parser = CustomOutputParser()

In [33]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [40]:
tools = get_tools("what is the phone number?")
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
)

In [41]:
tool_names

['retrieval', 'foo-9', 'foo-7', 'foo-6']

In [48]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True,
    max_iterations=2
)

In [49]:
agent_executor.run("what is the email addres of Alice?")



> Entering new AgentExecutor chain...
Thought: Ahoy, we need to search through the user's documents for Alice's email address
Action: retrieval
Action Input: "Alice email address"

Observation:This is Alice's phone number: 123-456-7890 Email: myemail@mail.com
Arrr, we found Alice's email address in the retrieval! But let's make sure it be correct.
Action: retrieval
Action Input: "Alice's email address"

Observation:This is Alice's phone number: 123-456-7890 Email: myemail@mail.com


> Finished chain.


'Agent stopped due to iteration limit or time limit.'